In [1]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import os
from dotenv import load_dotenv
from selenium.webdriver.common.keys import Keys
load_dotenv()

True

In [2]:
my_email = os.getenv("email")
my_password = os.getenv("password")

In [3]:
def extract_reddit_posts(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)

    # Wait for the feed to load
    WebDriverWait(driver, 10)
    
    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(6)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break


    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    posts = soup.find_all("div", {"class": "_1poyrkZ7g36PawDueRza-J"})

    rows = []
    for post in posts:
        link = post.find_all('a')
        try:
            linkurl = 'https://www.reddit.com' + re.search(r"/r/(\w+)/comments/(\w+)/(\w+)/", str(link)).group()
        except:
            linkurl = 'Error loading'
        post_title = post.find('h3', class_='_eYtD2XCVieq6emjKBH3m')
        rows.append([linkurl, post_title.text])
            
        rows.append([linkurl, post_title.text])

    driver.quit()

    return pd.DataFrame(rows, columns=['Link', "Title"])


In [22]:
def extract_reddit_post_info(df):
    reddit_comments = []

    for i, row in df.iterrows():
        url = row['Link']
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        #options.add_argument("--headless")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, "html.parser")

        post_container = soup.find("div", {"data-testid": "post-container"})

        try:
            title = post_container.find("h1", {"class": "_eYtD2XCVieq6emjKBH3m"})
            username_link = post_container.find("a", {"class": "_2tbHP6ZydRpjI44J3syuqC"})
            username = username_link.text.strip("u/")
            message = post_container.find("div", {"class": "_292iotee39Lmt0MkQZ2hPV RichTextJSON-root"}).text
            date = post_container.find("span", {"class": "_2VF2J19pUIMSLJFky-7PEI"}).text
            upvotes = post_container.find("div", {"class": "_1E9mcoVn4MYnuBQSVDt1gC"}).text
            comments = post_container.find("span", {"class": "FHCV02u6Cp2zYL0fhQPsO"}).text
            reddit_comments.append([title, username, message, date, upvotes, comments, url])
        except:
            pass

        driver.quit()

        print(f'{url} done')

    return pd.DataFrame(reddit_comments, columns=["Title", "Username", "Message", "Date", "Upvotes", "Comments", "URL"])


In [24]:

def extract_reddit_post_comments(df):
    reddit_post_comments = []

    for i, row in df.iterrows():
        url = row['Link']
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        #options.add_argument("--headless")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        time.sleep(5)

        while True:
            scroll_height = 10
            document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
            time.sleep(3)
            document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
            if document_height_after == document_height_before:
                break

        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, 'html.parser')
        driver.quit()
        posts = soup.find_all('div', {'class': '_3tw__eCCe7j-epNCKGXUKk'})

        for post in posts:
            try:
                name = post.find('a', {'class': 'wM6scouPXXsFDSZmZPHRo DjcdNGtVXPcxG0yiFXIoZ _23wugcdiaj44hdfugIAlnX'}).text
            except:
                name = post.find('a', {'class': 'wM6scouPXXsFDSZmZPHRo DjcdNGtVXPcxG0yiFXIoZ _23wugcdiaj44hdfugIAlnX'})

            try:
                date = post.find('a', class_='_3yx4Dn0W3Yunucf5sVJeFU').text
            except:
                date = post.find('a', class_='_3yx4Dn0W3Yunucf5sVJeFU')
            
            try:
                upvotes = post.find('div', {'class':"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M"}).text
            except:
                upvotes = post.find('div', {'class':"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M"})

            try:
                comment_thread = post.find('p', class_='_1qeIAgB0cPwnLhDF9XSiJM').text
            except:
                comment_thread = post.find('p', class_='_1qeIAgB0cPwnLhDF9XSiJM')
            reddit_post_comments.append([name, date, upvotes, comment_thread, url])

    df_reddit_post_comments2 = pd.DataFrame(reddit_post_comments, columns=["Name", 'date', 'upvotes', 'comment2', 'url'])
    return df_reddit_post_comments2




In [20]:
df_reddit1 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link')
df_reddit1.drop_duplicates(inplace=True)
df_reddit1.to_csv('df_reddit1.csv', index = False)


In [13]:
df_reddit2 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=comments')
df_reddit2.drop_duplicates(inplace=True)
df_reddit2.to_csv('df_reddit2.csv', index = False)


In [14]:
df_reddit3 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=new')
df_reddit3.drop_duplicates(inplace=True)
df_reddit3.to_csv('df_reddit3.csv', index = False)


In [15]:
df_reddit4 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=top')
df_reddit4.drop_duplicates(inplace=True)
df_reddit4.to_csv('df_reddit4.csv', index = False)


In [16]:
df_reddit5 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=hot')
df_reddit5.drop_duplicates(inplace=True)
df_reddit5.to_csv('df_reddit5.csv', index = False)


In [ ]:
df_reddit1 = pd.read_csv('df_reddit1.csv')
df_reddit2 = pd.read_csv('df_reddit2.csv')
df_reddit3 = pd.read_csv('df_reddit3.csv')
df_reddit4 = pd.read_csv('df_reddit4.csv')
df_reddit5 = pd.read_csv('df_reddit5.csv')

In [ ]:
df_reddit1 = df_reddit1[df_reddit1['Link'] != 'Error loading']
df_reddit2 = df_reddit2[df_reddit2['Link'] != 'Error loading']
df_reddit3 = df_reddit3[df_reddit3['Link'] != 'Error loading']
df_reddit4 = df_reddit4[df_reddit4['Link'] != 'Error loading']
df_reddit5 = df_reddit5[df_reddit5['Link'] != 'Error loading']


In [21]:
erpi1 = extract_reddit_post_info(df_reddit1)
erpi1.to_csv('erpi1.csv', index = False)

https://www.reddit.com/r/Futurology/comments/10qvt8l/chatgpt_is_just_the_beginning_artificial/ done


KeyError: 1

In [17]:
erpi2 =extract_reddit_post_info(df_reddit2)
erpi2.to_csv('erpi2.csv', index = False)

https://www.reddit.com/r/pcmasterrace/comments/wddcpe/giveaway_giving_away_10_deskmats_from_the_ai/ done


KeyError: 1

In [ ]:
erpi3 = extract_reddit_post_info(df_reddit3)
erpi3.to_csv('erpi3.csv', index = False)

In [ ]:
erpi4 = extract_reddit_post_info(df_reddit4)
erpi4.to_csv('erpi4.csv', index = False)

In [ ]:
erpi5 = extract_reddit_post_info(df_reddit5)
erpi5.to_csv('erpi5.csv', index = False)

In [ ]:
erpc1 = extract_reddit_post_comments(df_reddit1)
erpc1.to_csv('erpc1.csv', index = False)

,Name,date,upvotes,comment2,url
0,FuturologyBot,1 mo. ago,Vote,The following submission statement was provide...,https://www.reddit.com/r/Futurology/comments/1...
1,CaptPants,1 mo. ago,4.7k,I hope it's used for more than just cutting jo...,https://www.reddit.com/r/Futurology/comments/1...
2,Shanhaevel,1 mo. ago,2.0k,"Haha, that's rich. As if.",https://www.reddit.com/r/Futurology/comments/1...
3,SatiricalComment,1 mo. ago,1.1k,One of the intents of many scientists who deve...,https://www.reddit.com/r/Futurology/comments/1...
4,fistfulloframen,1 mo. ago,30,You can use it to fix up your resume after you...,https://www.reddit.com/r/Futurology/comments/1...
...,...,...,...,...,...
705,HairyChest69,14 days ago,1,I always wonder about retail workers. The peop...,https://www.reddit.com/r/Futurology/comments/1...
706,AnEggscellentName,14 days ago,None,None,https://www.reddit.com/r/Futurology/comments/1...
707,TheWaterDoctor,14 days ago,1,Building services and maintenance.,https://www.reddit.com/r/Futurology/comments/1...
708,Unrigg3D,14 days ago,1,Nobody is at risk for not having jobs. This is...,https://www.reddit.com/r/Futurology/comments/1...


In [25]:
erpc2 = extract_reddit_post_comments(df_reddit2)
erpc2.to_csv('erpc2.csv', index = False)

In [29]:
df_reddit3.head(15)

,Link,Title
0,https://www.reddit.com/r/cryptostreetbets/comm...,$CUMINU the crypto version of OnlyFans. OnlyFa...
2,https://www.reddit.com/r/aibiz/comments/11qgze...,Best AI Newsletters
4,https://www.reddit.com/r/businessanalysis/comm...,Need better ideas to sell edtech app to 1070 s...
6,https://www.reddit.com/r/DharmikEdits/comments...,Need better ideas to sell edtech app to 1070 s...
8,https://www.reddit.com/r/CryptoMoonShots/comme...,$CUMINU the crypto version of OnlyFans. OnlyFa...
10,https://www.reddit.com/r/sweatystartup/comment...,Need better ideas to sell edtech app to 1070 s...
12,https://www.reddit.com/r/IndiaSpeaks/comments/...,Need better ideas to sell edtech app to 1070 s...
14,https://www.reddit.com/r/EntrepreneurRideAlong...,Need better ideas to sell edtech app to 1070 s...
16,https://www.reddit.com/r/startup_resources/com...,Need better ideas to sell edtech app to 1070 s...
18,https://www.reddit.com/r/SideProject/comments/...,Need better ideas to sell edtech app to 1070 s...


In [30]:
erpc3 = extract_reddit_post_comments(df_reddit3)
erpc3.to_csv('erpc3.csv', index = False)

In [ ]:
erpc4 = extract_reddit_post_comments(df_reddit4)
erpc4.to_csv('erpc4.csv', index = False)

In [ ]:
erpc5 = extract_reddit_post_comments(df_reddit5)
erpc5.to_csv('erpc5.csv', index = False)

In [ ]:
erpc6 = extract_reddit_post_comments(df_reddit1)
erpc6.to_csv('erpc5.csv', index = False)